<a href="https://colab.research.google.com/github/isa-pinheiro/Learning-From-Data/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hoeffding Inequality
Run a computer simulation for flipping 1,000 virtual fair coins. Flip each coin independently 10 times. Focus on 3 coins as follows: c1 is the first coin flipped, crand is a coin chosen randomly from the 1,000, and cmin is the coin which had the minimum frequency of heads (pick the earlier one in case of a tie). Let ν1, νrand, and νmin be the fraction of heads obtained for the 3 respective coins out of the 10 tosses.

Run the experiment 100,000 times in order to get a full distribution of ν1, vrand, and νmin (note that crand and cmin will change from run to run)

In [ ]:
import random
import numpy as np

In [ ]:
# 1 represents heads and 0 represents tails
# função de criar as moedas
# retornar a frequencia

def flip_coin(n):
    state = np.empty((0,n))

    for _ in range(10):
        state = np.append(state, random.randint(0,1))

    v = (state.sum())/n

    return v

In [ ]:
# cria as 1000 moedas

def create_n_coins(n_flips, n_coins):
    v = np.empty((0, n_coins))

    for _ in range(n_coins):
        v = np.append(v, flip_coin(n_flips))

    v1 = v[0]
    vrand = v[random.randint(0,n_coins-1)]
    vmin = v.min()
    # print(v)
    # print(v1, vrand, vmin)
    return v1, vrand, vmin

In [ ]:
total_v1, total_vrand, total_vmin = [], [], []

for i in range(100000):
    # print(i)
    v1, vrand, vmin = create_n_coins(10, 1000)
    total_v1.append(v1)
    total_vrand.append(vrand)
    total_vmin.append(vmin)

    if i == 0:
        actual_v1 = v1

In [ ]:
print('média v1', np.mean(total_v1))
print('média vrand', np.mean(total_vrand))
print('média vmin', np.mean(total_vmin))

média v1 0.4945
média vrand 0.49639999999999995
média vmin 0.039599999999999996


# Linear Regression
In these problems, we will explore how Linear Regression for classification works. As
with the Perceptron Learning Algorithm in Homework # 1, you will create your own
target function f and data set D. Take d = 2 so you can visualize the problem, and
assume X = [−1, 1] × [−1, 1] with uniform probability of picking each x ∈ X . In
each run, choose a random line in the plane as your target function f (do this by
taking two random, uniformly distributed points in [−1, 1] × [−1, 1] and taking the
line passing through them), where one side of the line maps to +1 and the other maps
to −1. Choose the inputs xn of the data set as random points (uniformly in X ), and
evaluate the target function on each xn to get the corresponding output yn.

In [ ]:
def create_data(N_train, N_test):
    # y = ax +b
    # p = (x, y)

    p1 = np.random.uniform(-1, 1, 2)
    p2 = np.random.uniform(-1, 1, 2)

    a = (p1[1] - p2[1]) / (p1[0] - p2[0])
    b = p1[1] - a * p1[0]

    X_train = np.random.uniform(-1, 1, (N_train, 2))
    y_train = np.sign((a * X_train[:, 0] + b) - X_train[:, 1])

    X_test = np.random.uniform(-1, 1, (N_test, 2))
    y_test = np.sign((a * X_test[:, 0] + b) - X_test[:, 1])

    X_train = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
    X_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1)
    return X_train, y_train, X_test, y_test, a, b



In [ ]:
def missed_index(y_true, y_pred):
    mis_i = []
    for i in range(y_true.shape[0]):
        if y_true[i] != y_pred[i]:
            mis_i.append(i)
    return mis_i

In [ ]:
def perceptron(X_train, X_test, y_train, y_test, w = np.zeros(3)):
    total_iter = []
    total_error = []
    for _ in range(1000):
        iter = 0

        while True:
            iter += 1

            y_pred_train = np.sign(np.dot(X_train, w.T))

            mis_i_train = missed_index(y_train, y_pred_train)

            if len(mis_i_train) == 0:
                break

            rand_i = mis_i_train[random.randint(0, len(mis_i_train) - 1)]

            w = w + y_train[rand_i] * X_train[rand_i]

        total_iter.append(iter)

        # calculo do erro
        y_pred_test = np.sign(np.dot(X_test, w.T))

        mis_i_test = missed_index(y_test, y_pred_test)

        error = len(mis_i_test)/1000

        total_error.append(error)
    return np.mean(total_iter), np.mean(total_error)

In [ ]:
def linear_regression(N_train, N_test):
    total_error_train = []
    total_error_test = []

    for _ in range(1000):
        X_train, y_train, X_test, y_test, a, b = create_data(N_train, N_test)

        X_train_pinv = np.linalg.pinv(X_train)
        w = np.dot(X_train_pinv, y_train)

        y_pred_train = np.sign(np.dot(X_train, w.T))

        mis_i_train = missed_index(y_train, y_pred_train)

        error_train = len(mis_i_train)/N_train

        total_error_train.append(error_train)

        # calculo do erro
        y_pred_test = np.sign(np.dot(X_test, w.T))

        mis_i_test = missed_index(y_test, y_pred_test)

        error_test = len(mis_i_test)/N_test

        total_error_test.append(error_test)

    return np.mean(total_error_train), np.mean(total_error_test)

In [ ]:
def linear_regressionwpla(N_train, N_test):
    total_error_train = []
    total_error_test = []

    for _ in range(1000):
        X_train, y_train, X_test, y_test, a, b = create_data(N_train, N_test)

        X_train_pinv = np.linalg.pinv(X_train)
        w = np.dot(X_train_pinv, y_train)

        y_pred_train = np.sign(np.dot(X_train, w.T))

        mis_i_train = missed_index(y_train, y_pred_train)

        error_train = len(mis_i_train)/N_train

        total_error_train.append(error_train)

        # calculo do erro
        y_pred_test = np.sign(np.dot(X_test, w.T))

        mis_i_test = missed_index(y_test, y_pred_test)

        error_test = len(mis_i_test)/N_test

        total_error_test.append(error_test)

        iter_pla, error_pla = perceptron(X_train, X_test, y_train, y_test, w)
    return np.mean(total_error_train), np.mean(total_error_test), iter_pla, error_pla

In [ ]:
# Questões 5 e 6
total_error_train, total_error_test = linear_regression(N_train = 100, N_test = 1000)
print('media de erro no treino', total_error_train)
print('media de erro no teste', total_error_test)

media de erro no treino 0.038680000000000006
media de erro no teste 0.048196


In [ ]:
_, _, iter_pla, error_pla = linear_regressionwpla(N_train = 10, N_test = 1000)
print('media de iterações', iter_pla)
print('media de erro', error_pla)

media de iterações 1.0
media de erro 0.08899999999999998


# Nonlinear Transformation
In these problems, we again apply Linear Regression for classification. Consider the
target function: \\
 \\
f(x1, x2) = sign(x1^2 + x2^2 − 0.6) \\
 \\
Generate a training set of N = 1000 points on X = [−1, 1] × [−1, 1] with a uniform
probability of picking each x ∈ X . Generate simulated noise by flipping the sign of
the output in a randomly selected 10% subset of the generated training set.

In [ ]:
def create_noisy_data(N_train, N_test):
    # y = ax +b
    # p = (x, y)
    X_train = np.random.uniform(-1, 1, (N_train, 2))
    y_train = np.sign(np.square(X_train[:, 0]) + np.square(X_train[:, 1]) - 0.6)

    X_test = np.random.uniform(-1, 1, (N_test, 2))
    y_test = np.sign(np.square(X_test[:, 0]) + np.square(X_test[:, 1]) - 0.6)

    rand_i_train = np.random.choice(y_train.size, size=(int(len(y_train) * 0.1)), replace=False)
    y_train[rand_i_train] = -y_train[rand_i_train]

    rand_i_test = np.random.choice(y_test.size, size=(int(len(y_test) * 0.1)), replace=False)
    y_test[rand_i_test] = -y_test[rand_i_test]

    X_train = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
    X_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1)

    return X_train, y_train, X_test, y_test


In [ ]:
# (1, x1, x2, x1x2, x1^2, x2^2)

def transform(X):
    Z = np.zeros((X.shape[0], 6))
    Z[:, 0] = 1
    Z[:, 1] = X[:, 1]
    Z[:, 2] = X[:, 2]
    Z[:, 3] = X[:, 1] * X[:, 2]
    Z[:, 4] = np.square(X[:, 1])
    Z[:, 5] = np.square(X[:, 2])
    return Z


In [ ]:
def linear_regression_noisy(N_train, N_test, will_transform = False):
    total_error_train = []
    total_error_test = []
    all_w = []
    for _ in range(1000):
        X_train, y_train, X_test, y_test = create_noisy_data(N_train, N_test)
        if transform:
            X_train = transform(X_train)
            X_test = transform(X_test)

        X_train_pinv = np.linalg.pinv(X_train)
        w = np.dot(X_train_pinv, y_train)

        all_w.append(w)
        y_pred_train = np.sign(np.dot(X_train, w.T))

        mis_i_train = missed_index(y_train, y_pred_train)

        error_train = len(mis_i_train)/N_train

        total_error_train.append(error_train)

        # calculo do erro
        y_pred_test = np.sign(np.dot(X_test, w.T))

        mis_i_test = missed_index(y_test, y_pred_test)

        error_test = len(mis_i_test)/N_test

        total_error_test.append(error_test)

    return np.mean(total_error_train), np.mean(total_error_test), all_w

In [ ]:
N_train = 1000
N_test = 1000

In [ ]:
# Questão 08
error_mean_train, error_mean_test, _ = linear_regression_noisy(N_train, N_test)
error_mean_train

0.12399700000000001

In [ ]:
transf_error_mean_train, transf_error_mean_test, w_mean = linear_regression_noisy(N_train, N_test, True)

In [ ]:
transf_error_mean_train

0.12372

In [ ]:
w_mean = np.mean(w_mean, axis=0)
[f"{w:.6f}" for w in w_mean]

['-0.992470', '0.002163', '-0.000701', '0.001885', '1.559744', '1.558534']